# Logger ユーティリティ

[api 一覧はこちら](./manual.ipynb)

Botの動作中の状況を保存するクラスです。

RunnerでAgentを動かしたあとに得られるセッションクラスからlogプロパティで取り出すことができます。
また、Runnerで実行時にログファイルに保存しておいて、あとからロードして分析することもできます。

以下はファイルを読み込むサンプルです。

In [ ]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。
! pip install --upgrade rbot

In [ ]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

In [ ]:
from rbot import Logger

import polars as pl

In [ ]:
logger = Logger()               # Loggerオブジェクトの作成

logger.restore("./bot.log")    # サンプルログファイルの読み込み
                                #(サンプルログファイル`bot.log`はgithubの同じディレクトリにおいてあります)

## オーダー情報の取り出し

`Logger.orders`プロパティでオーダー状況が取り出せます。各カラムの説明は以下。

#### オーダー概要
* `log_id`  ログの通し番号
* `symbol`  取引通過ペアー
* `update_time` 更新時間
* `create_time` 作成時間
* `status`  オーダー状況（以下の値を取る）
  * `New`　新規
  * `PartiallyFilled`　一部約定
  * `Filled`　約定済
  * `Canceled`　キャンセル

#### オーダー情報（作成時の情報が維持される）
* `order_id`　オーダーID
* `client_order_id` クライアント作成のID
* `order_side` 売り・買い種別（以下の値を取る）
  * `Buy`
  * `Sell`
* `order_type`　オーダ種別（以下の値を取る）
  * `Limit`
  * `Market`
* `order_price`　指値
* `order_size`　注文サイズ

#### トランザクション情報（トランザクション結果が反映される）
* `transaction_id`　トランザクションID
* `execute_price` 約定価格
* `execute_size`　約定サイズ
* `quote_vol`　見積もりサイズ（ホーム側のサイズ）
* `commission`　手数料
* `commistion_asset`　手数料通貨
* `is_maker`　Makerの場合はTrue
* `message` オーダメッセージ（未実装）
* `commision_home`　ホーム通貨での手数料
* `commision_foreign`　フォーリン通貨での手数料

#### アカウント情報（オーダーによって生じた残高の変化[差分]）
* `home_change` ホーム残高の変化
* `foreign_change`　フォーリン残高の変化
* `free_home_change`　ホームの利用可能残高の変化
* `free_foreign_change`　フォーリンの利用残高の変化
* `lock_home_change`　ホームのロック（板に置いた状態）残高の変化
* `lock_foreign_change`　フォーリンのロック（板に置いた状態）残高の変化

#### セッション開始からの仮想ポジション・損益（現物にはポジションはないが仮想的に計算）
* `open_position`　オープンされたポジション数
* `close_position`　クローズされたポジション数
* `position`　ポジションサイズ
* `profit`　利益（手数料含まず）
* `fee`　手数料
* `total_profit`　手数料を含む利益
* `sum_profit`　セッション開始からの`total_profit`の累積和


In [ ]:
logger.orders.head(5)

## インジケータの取り出し

Agentから`session.log_indicator("key", value)`で保存したインジケータを取り出すことができます。Logger[`key`]で取り出せます。取り出したデータはPolarsのDataFrame形式になっています。

In [ ]:
# session.log_indicator("range", 123.4) で記録した指標を取得。（Key,値はサンプル）

logger['range'].head(5)